In [1]:
import pandas as pd

# Create Data File

The data source is https://console.cloud.google.com/marketplace/details/metabrainz/listenbrainz 

We pick the top top 300 most popular artists by number of listens. The data is available thanks to BigQuery's 1TB/mo of free tier processing.

The first step is we query the data set:


In [2]:
play_counts_df = pd.read_feather("play_counts_df.feather")

In [3]:
play_counts_df.head(10)

,user_name,artist_name,cnt
0,munhoz,Daft Punk,663
1,enkadeze,U2,321
2,Kilu,U2,32
3,ioncewasacat,Air,551
4,toastpaint,Air,342
5,Boris_Neo,Air,190
6,jaffry,Air,1041
7,ars_mvsica,Air,931
8,I_am_walking,Air,158
9,Svarthjelm,Air,32


## group by user by pivot the table

The value is the number of listens for each user per artist:


In [4]:
play_counts_pivot_df = play_counts_df.pivot(index='user_name', columns='artist_name', values='cnt')
play_counts_pivot_df.head()

artist_name,30 Seconds to Mars,65daysofstatic,A Perfect Circle,A Tribe Called Quest,ABBA,AC/DC,Adele,Aerosmith,Air,Alanis Morissette,...,ZZ Top,alt-J,blink-182,deadmau5,久石譲,川井憲次,梶浦由記,植松伸夫,菅野よう子,近藤浩治
user_name,,,,,,,,,,,,,,,,,,,,,
-andor-,<NA>,46,<NA>,43,7,13,<NA>,4,71,<NA>,...,12,<NA>,<NA>,29,<NA>,1,<NA>,<NA>,<NA>,<NA>
-dAtA-TRoN-,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
-nils-,<NA>,2,49,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,8,<NA>,<NA>,<NA>,<NA>,12,<NA>,<NA>,<NA>,<NA>
00dani,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,3,<NA>,<NA>,<NA>,<NA>,<NA>,8,<NA>,10
00void,3,425,1,<NA>,<NA>,1,<NA>,12,17,<NA>,...,6,<NA>,11,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


This is a quick way to flatten the pivoted table:

In [5]:
import pandas as pd
df_out = pd.DataFrame(play_counts_pivot_df.to_records())
df_out.head(4)

,user_name,30 Seconds to Mars,65daysofstatic,A Perfect Circle,A Tribe Called Quest,ABBA,AC/DC,Adele,Aerosmith,Air,...,ZZ Top,alt-J,blink-182,deadmau5,久石譲,川井憲次,梶浦由記,植松伸夫,菅野よう子,近藤浩治
0,-andor-,<NA>,46,<NA>,43,7,13,<NA>,4,71,...,12,<NA>,<NA>,29,<NA>,1,<NA>,<NA>,<NA>,<NA>
1,-dAtA-TRoN-,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,-nils-,<NA>,2,49,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,8,<NA>,<NA>,<NA>,<NA>,12,<NA>,<NA>,<NA>,<NA>
3,00dani,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,3,<NA>,<NA>,<NA>,<NA>,<NA>,8,<NA>,10


In [6]:
df_out[df_out.user_name == "-nils-"].iloc[0]['Muse']

20

## Enrich Data

We will enrich the data from [this notebook: Enrich_top_300.ipynb](Enrich_top_300.ipynb).

The idea here is we create a tag counter. So for each of the 300 artists we create or increment the column for that croud. f three of the artists played by the contain the "rock" tag, then the value for that user tag_rock column is 3, if 4 artists have a 80s tag, then tag_80s is 4, and so on...



In [7]:
band_tags = pd.read_csv("bands_tags.csv")
band_tags.head()

,Unnamed: 0,artist_name,tag_alternative,tag_alternative rock,tag_rock,tag_indie,tag_electronic,tag_classic rock,tag_british,tag_60s,...,tag_jay z,tag_shoegazer,tag_hair metal,tag_rapcore,tag_underground hip hop,tag_symphonic black metal,tag_darkwave,tag_world,tag_latin,tag_spanish
0,0,Radiohead,True,True,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,The Beatles,False,False,True,False,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,2,Pink Floyd,False,False,True,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,3,Daft Punk,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,4,Muse,True,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [8]:
list(band_tags.columns)[:15]

['Unnamed: 0',
 'artist_name',
 'tag_alternative',
 'tag_alternative rock',
 'tag_rock',
 'tag_indie',
 'tag_electronic',
 'tag_classic rock',
 'tag_british',
 'tag_60s',
 'tag_pop',
 'tag_progressive rock',
 'tag_psychedelic rock',
 'tag_psychedelic',
 'tag_house']

following may take awhile to run. This goes over every row in the user list (df_out) and compares every band to all the appropriate column, if it finds that that person listed to that band, it increments each tag for that artists: showing preference for those generes. 

In [9]:
from collections import defaultdict

tag_for = {}
for xi, xrow in df_out.iterrows():
    tag_for[xi] = defaultdict(int)
    for index, row in band_tags.iterrows():
        if pd.notnull(xrow[row['artist_name']]):
            if xrow[row['artist_name']] > 0:
                tags = [name.lower() for name, val in row.iteritems() if "tag_" in name and val == True]
                for tag in tags:
                    tag_for[xi][tag] += 1                          

 

Add the tags with their counts to the DataFrame as additional features

In [10]:
for xi, values in tag_for.items():
    for tag_name, count in values.items():
        df_out.at[xi, tag_name] = count
        
df_out.head().to_csv("test.csv", index=False)
df_out.head()   

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


,user_name,30 Seconds to Mars,65daysofstatic,A Perfect Circle,A Tribe Called Quest,ABBA,AC/DC,Adele,Aerosmith,Air,...,tag_melodic metal,tag_halo,tag_game soundtracks,tag_symphonic black metal,tag_underground hip hop,tag_grindcore,tag_noisecore,tag_noisegrind,tag_comedy,tag_monkey island
0,-andor-,<NA>,46,<NA>,43,7,13,<NA>,4,71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-dAtA-TRoN-,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-nils-,<NA>,2,49,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00dani,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00void,3,425,1,<NA>,<NA>,1,<NA>,12,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Set a Target column to turn this into a classification problem, if they play Beatles (even once) then label is True

In [11]:
import numpy as np
df_out['Like The Beatles'] =  df_out['The Beatles'].apply(lambda x: not pd.isnull(x))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


We need to clean up the column names as Cloud AutoML is very picky and will fail if you do things like have Unicode file names. I wish Google will fix this. I'll complain ;)

In [12]:
import csv

df_out.columns = df_out.columns.str.replace("post-roc", "post roc1")
df_out.columns = df_out.columns.str.replace("trip-hop", "trip hop1")
df_out.columns = df_out.columns.str.replace(" ", "_")
df_out.columns = df_out.columns.str.replace("/", "")
df_out.columns = df_out.columns.str.replace("é", "")
df_out.columns = df_out.columns.str.replace("ö", "o")
df_out.columns = df_out.columns.str.replace("+", "_and_")
df_out.columns = df_out.columns.str.replace("&", "and")
df_out.columns = df_out.columns.str.replace("!", "")
df_out.columns = df_out.columns.str.replace("-", "_")
df_out.columns = df_out.columns.str.replace(".", "")
df_out.columns = df_out.columns.str.replace("ó", "o")
df_out.columns = df_out.columns.str.replace("'", "")

# Japanese Artists
df_out.columns = df_out.columns.str.replace("久石譲", "Joe_Hisaishi")
df_out.columns = df_out.columns.str.replace("川井憲次", "Kenji_Kawai")
df_out.columns = df_out.columns.str.replace("梶浦由記", "Yuki_Kajiura")
df_out.columns = df_out.columns.str.replace("植松伸夫", "Nobuo_Uematsu")
df_out.columns = df_out.columns.str.replace("菅野よう子", "Yoko_Kanno")
df_out.columns = df_out.columns.str.replace("近藤浩治", "Koji_Kondo")


df_out.to_csv("file_out_2495_tags.csv", index=False)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  if __name__ == "__main__":
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  del sys.path[0]


In [13]:
!head file_out_2495_tags.csv

user_name,30_Seconds_to_Mars,65daysofstatic,A_Perfect_Circle,A_Tribe_Called_Quest,ABBA,ACDC,Adele,Aerosmith,Air,Alanis_Morissette,Alice_in_Chains,Amon_Amarth,Amon_Tobin,Amorphis,Anal_Cunt,Anathema,Animal_Collective,Aphex_Twin,Apocalyptica,Arcade_Fire,Arctic_Monkeys,Audioslave,Autechre,Avenged_Sevenfold,Avril_Lavigne,Ayreon,Bad_Religion,Beach_House,Beastie_Boys,Beck,Belle_and_Sebastian,Beyonc,Bjork,Black_Sabbath,Blind_Guardian,Bloc_Party,Blur,Boards_of_Canada,Bob_Dylan,Bob_Marley_and_The_Wailers,Bon_Iver,Bon_Jovi,Bonobo,Brand_New,Breaking_Benjamin,Brian_Eno,Britney_Spears,Broken_Social_Scene,Bruce_Springsteen,Burial,Bush,CHVRCHES,Cake,Calvin_Harris,Carly_Rae_Jepsen,Childish_Gambino,Children_of_Bodom,Cocteau_Twins,Coheed_and_Cambria,Coldplay,Cradle_of_Filth,Creedence_Clearwater_Revival,Crystal_Castles,DJ_Shadow,Daft_Punk,Dark_Tranquillity,Dave_Matthews_Band,David_Bowie,Dead_Can_Dance,Death_Cab_for_Cutie,Death_Grips,Deep_Purple,Deftones,Depeche_Mode,Dire_Straits,Disturbed,Drake,Dream_Thea